# BedrockEmbeddings

[Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI.

This will help you get started with Amazon Bedrock [embedding models](/docs/concepts/embedding_models) using LangChain. For detailed documentation on `Bedrock` features and configuration options, please refer to the [API reference](https://api.js.langchain.com/classes/langchain_aws.BedrockEmbeddings.html).

## Overview
### Integration details

| Class | Package | Local | [Py support](https://python.langchain.com/docs/integrations/text_embedding/bedrock/) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [Bedrock](https://api.js.langchain.com/classes/langchain_aws.BedrockEmbeddings.html) | [@langchain/aws](https://api.js.langchain.com/modules/langchain_aws.html) | ❌ | ✅ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/aws?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/aws?style=flat-square&label=%20&) |

## Setup

To access Bedrock embedding models you'll need to create an AWS account, get an API key, and install the `@langchain/aws` integration package.

Head to the [AWS docs](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html) to sign up for AWS and setup your credentials. You'll also need to turn on model access for your account, which you can do by [following these instructions](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html).

### Credentials

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```bash
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="your-api-key"
```

### Installation

The LangChain Bedrock integration lives in the `@langchain/aws` package:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/aws @langchain/core
</Npm2Yarn>
```

## Instantiation

Now we can instantiate our model object and embed text.

There are a few different ways to authenticate with AWS - the below examples rely on an access key, secret access key and region set in your environment variables:

In [1]:
import { BedrockEmbeddings } from "@langchain/aws";

const embeddings = new BedrockEmbeddings({
  region: process.env.BEDROCK_AWS_REGION!,
  credentials: {
    accessKeyId: process.env.BEDROCK_AWS_ACCESS_KEY_ID!,
    secretAccessKey: process.env.BEDROCK_AWS_SECRET_ACCESS_KEY!,
  },
  model: "amazon.titan-embed-text-v1",
});

## Indexing and Retrieval

Embedding models are often used in retrieval-augmented generation (RAG) flows, both as part of indexing data as well as later retrieving it. For more detailed instructions, please see our RAG tutorials under the [working with external knowledge tutorials](/docs/tutorials/#working-with-external-knowledge).

Below, see how to index and retrieve data using the `embeddings` object we initialized above. In this example, we will index and retrieve a sample document using the demo [`MemoryVectorStore`](/docs/integrations/vectorstores/memory).

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## Direct Usage

Under the hood, the vectorstore and retriever implementations are calling `embeddings.embedDocument(...)` and `embeddings.embedQuery(...)` to create embeddings for the text(s) used in `fromDocuments` and the retriever's `invoke` operations, respectively.

You can directly call these methods to get embeddings for your own use cases.

### Embed single texts

You can embed queries for search with `embedQuery`. This generates a vector representation specific to the query:

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
         0.625,  0.111328125,      0.265625,   -0.20019531,  0.40820312,
  -0.010803223,  -0.22460938, -0.0002937317,    0.29882812, -0.14355469,
  -0.068847656,   -0.3984375,          0.75,    -0.1953125,  -0.5546875,
  -0.087402344,       0.5625,      1.390625,    -0.3515625,  0.39257812,
  -0.061767578,      0.65625,   -0.36328125,   -0.06591797,    0.234375,
   -0.36132812,   0.42382812,  -0.115234375,   -0.28710938, -0.29296875,
     -0.765625,  -0.16894531,    0.23046875,     0.6328125, -0.08544922,
    0.13671875, 0.0004272461,        0.3125,    0.12207031,   -0.546875,
    0.14257812, -0.119628906,  -0.111328125,    0.61328125,      0.6875,
     0.3671875,   -0.2578125,   -0.27734375,      0.703125,    0.203125,
    0.17675781,  -0.26757812,   -0.76171875,    0.71484375,  0.77734375,
    -0.1953125, -0.007232666,  -0.044921875,    0.23632812, -0.24121094,
  -0.012207031,    0.5078125,    0.08984375,    0.56640625,  -0.3046875,
     0.6484375,        -0.25,   -0.37890625,    -

### Embed multiple texts

You can embed multiple texts for indexing with `embedDocuments`. The internals used for this method may (but do not have to) differ from embedding queries:

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
         0.625,  0.111328125,      0.265625,   -0.20019531,  0.40820312,
  -0.010803223,  -0.22460938, -0.0002937317,    0.29882812, -0.14355469,
  -0.068847656,   -0.3984375,          0.75,    -0.1953125,  -0.5546875,
  -0.087402344,       0.5625,      1.390625,    -0.3515625,  0.39257812,
  -0.061767578,      0.65625,   -0.36328125,   -0.06591797,    0.234375,
   -0.36132812,   0.42382812,  -0.115234375,   -0.28710938, -0.29296875,
     -0.765625,  -0.16894531,    0.23046875,     0.6328125, -0.08544922,
    0.13671875, 0.0004272461,        0.3125,    0.12207031,   -0.546875,
    0.14257812, -0.119628906,  -0.111328125,    0.61328125,      0.6875,
     0.3671875,   -0.2578125,   -0.27734375,      0.703125,    0.203125,
    0.17675781,  -0.26757812,   -0.76171875,    0.71484375,  0.77734375,
    -0.1953125, -0.007232666,  -0.044921875,    0.23632812, -0.24121094,
  -0.012207031,    0.5078125,    0.08984375,    0.56640625,  -0.3046875,
     0.6484375,        -0.25,   -0.37890625,    -

## Configuring the Bedrock Runtime Client

You can pass in your own instance of the `BedrockRuntimeClient` if you want to customize options like
`credentials`, `region`, `retryPolicy`, etc.

In [ ]:
import { BedrockRuntimeClient } from "@aws-sdk/client-bedrock-runtime";
import { BedrockEmbeddings } from "@langchain/aws";

const getCredentials = () => {
  // do something to get credentials
}

// @lc-ts-ignore
const client = new BedrockRuntimeClient({
  region: "us-east-1",
  credentials: getCredentials(),
});

const embeddingsWithCustomClient = new BedrockEmbeddings({
  client,
});

## API reference

For detailed documentation of all Bedrock features and configurations head to the API reference: https://api.js.langchain.com/classes/langchain_aws.BedrockEmbeddings.html